# SQL Project: Answering Business Questions From Chinook DB

In this project we will use the Chinook database. Database and schema are attached to this jupiter file for reference.

The goal is to extract key insights of the employees, consumers and product sold by combining and filtering the different tables of the database.

Typical business questions we intend to answer:
- Consumer segmentation based on spend and/or location.
- Top-sellers and insights.
- Sell-out analysis: frequency, spend and category.
- Employee performance.
- Category analysis (track, playlists, etc.) 

We start loading the database:

In [3]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

Get a list of the tables in the database

In [3]:
%%sql
SELECT name, type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


From previous work, a query that gathers data about the invoice with an invoice_id of 4

In [3]:
%%sql

SELECT
    il.track_id,
    t.name AS track_name,
    ar.name AS artist_name,
    mt.name AS track_type,
    il.unit_price,
    il.quantity
FROM invoice_line il
INNER JOIN track AS t 
    ON t.track_id = il.track_id
INNER JOIN media_type AS mt 
    ON mt.media_type_id = t.media_type_id
INNER JOIN album AS a 
    ON a.album_id = t.album_id
INNER JOIN artist AS ar 
    ON ar.artist_id = a.artist_id
WHERE il.invoice_id = 4;

Done.


track_id,track_name,artist_name,track_type,unit_price,quantity
3448,"Lamentations of Jeremiah, First Set \ Incipit Lamentatio",The King's Singers,Protected AAC audio file,0.99,1
2560,Violent Pornography,System Of A Down,MPEG audio file,0.99,1
3336,War Pigs,Cake,Purchased AAC audio file,0.99,1
829,Let's Get Rocked,Def Leppard,MPEG audio file,0.99,1
1872,Attitude,Metallica,MPEG audio file,0.99,1
748,Dealer,Deep Purple,MPEG audio file,0.99,1
1778,You're What's Happening (In The World Today),Marvin Gaye,MPEG audio file,0.99,1
2514,Spoonman,Soundgarden,MPEG audio file,0.99,1


# Top Albums

A query that returns the top 5 albums, as calculated by the number of times a track from that album has been purchased. Your query should be sorted from most tracks purchased to least tracks purchased and return the following columns, in order:
- album, the title of the album artist, 
- the artist who produced the album
- tracks_purchased the total number of tracks purchased from that album

In [4]:
%%sql

SELECT 
    ta.album AS album,
    ta.artist_name AS artist,
    COUNT(*) AS tracks_purchased
FROM invoice_line AS il
INNER JOIN (
        SELECT 
            t.track_id, 
            ar.name AS artist_name,
            al.title AS album
        FROM track AS t
        INNER JOIN album AS al ON al.album_id=t.album_id
        INNER JOIN artist AS ar ON ar.artist_id=al.artist_id
            ) AS ta ON ta.track_id = il.track_id
GROUP BY album
ORDER BY 3 DESC
LIMIT 5


Done.


album,artist,tracks_purchased
Are You Experienced?,Jimi Hendrix,187
Faceless,Godsmack,96
Mezmerize,System Of A Down,93
Get Born,JET,90
The Doors,The Doors,83


# Staff Overview

A query that returns information about each employee and their supervisor using recursive joins.

In [9]:
%%sql

SELECT (e1.first_name ||" "|| e1.last_name) AS employee_name,
    e1.title AS employee_title,
    (e2.first_name ||" "|| e2.last_name) AS supervisor_name,
    e2.title AS supervisor_title
FROM employee as e1
LEFT JOIN employee AS e2 
    ON e1.reports_to = e2.employee_id
ORDER BY employee_name ASC;

Done.


employee_name,employee_title,supervisor_name,supervisor_title
Andrew Adams,General Manager,None,None
Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
Laura Callahan,IT Staff,Michael Mitchell,IT Manager
Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager
Michael Mitchell,IT Manager,Andrew Adams,General Manager
Nancy Edwards,Sales Manager,Andrew Adams,General Manager
Robert King,IT Staff,Michael Mitchell,IT Manager
Steve Johnson,Sales Support Agent,Nancy Edwards,Sales Manager


A query that finds the contact details of a customer with a first_name containing Belle from the database using pattern matching.

In [6]:
%%sql

SELECT
    first_name,
    last_name,
    phone
FROM customer
WHERE first_name LIKE "%Belle%";

Done.


first_name,last_name,phone
Isabelle,Mercier,+33 03 80 73 66 99


# Consumer Segmentation

Query that summarizes the purchases of each customer.

In [7]:
%%sql

SELECT
   c.first_name || " " || c.last_name AS customer_name,
   COUNT(i.invoice_id) AS number_of_purchases,
   ROUND(SUM(i.total),2) AS total_spent,
   CASE
       WHEN sum(i.total) < 40 THEN 'small spender'
       WHEN sum(i.total) > 100 THEN 'big spender'
       ELSE 'regular'
       END
       AS customer_category
FROM invoice i
INNER JOIN customer AS c 
    ON i.customer_id = c.customer_id
GROUP BY 1 ORDER BY 1;

Done.


customer_name,number_of_purchases,total_spent,customer_category
Aaron Mitchell,8,70.29,regular
Alexandre Rocha,10,69.3,regular
Astrid Gruber,9,69.3,regular
Bjørn Hansen,9,72.27,regular
Camille Bernard,9,79.2,regular
Daan Peeters,7,60.39,regular
Dan Miller,12,95.04,regular
Diego Gutiérrez,5,39.6,small spender
Dominique Lefebvre,9,72.27,regular
Eduardo Martins,12,60.39,regular


# Analysis of Playlists

A query that shows summary data for every playlist in the Chinook database. Use a WITH cluse to structure the query

In [9]:
%%sql

WITH playlist_info AS
    (
      SELECT
        p.playlist_id,
        p.name AS playlist_name,
        t.name AS track_name,
        (t.milliseconds/1000) AS length_seconds
     FROM playlist AS p
     LEFT JOIN playlist_track AS pt 
        ON pt.playlist_id = p.playlist_id
     LEFT JOIN track AS t 
        ON t.track_id = pt.track_id   
    )

SELECT playlist_id, playlist_name, 
    COUNT(track_name) AS number_of_tracks, 
    CAST(SUM(length_seconds) AS INT) AS length_seconds
FROM playlist_info
GROUP BY playlist_id
ORDER BY playlist_id

Done.


playlist_id,playlist_name,number_of_tracks,length_seconds
1,Music,3290,876049
2,Movies,0,None
3,TV Shows,213,500987
4,Audiobooks,0,None
5,90’s Music,1477,397970
6,Audiobooks,0,None
7,Movies,0,None
8,Music,3290,876049
9,Music Videos,1,294
10,TV Shows,213,500987


# Analysis of Customers

A query that works out how many customers that are in the USA and have purchased more than $90 are assigned to each sales support agent.

In [23]:
%%sql

WITH
    customer_usa AS
        (
        SELECT * FROM customer
        WHERE country = "USA"
        ),
    customer_gt_90_dollars AS 
        (
        SELECT c.*
        FROM invoice AS i
        INNER JOIN customer AS c 
            ON i.customer_id = c.customer_id
        GROUP BY 1
        HAVING SUM(i.total) > 90
        ),
    customers_usa_gt_90 AS
        (
         SELECT * FROM customer_usa
         INTERSECT
         SELECT * FROM customer_gt_90_dollars
        )

SELECT
    (e.first_name || " " || e.last_name) AS employee_name,
    COUNT(c.customer_id) AS customers_usa_gt_90
FROM employee e
LEFT JOIN customers_usa_gt_90 AS c 
    ON c.support_rep_id = e.employee_id
WHERE e.title = 'Sales Support Agent'
GROUP BY 1 ORDER BY 1;

Done.


employee_name,customers_usa_gt_90
Jane Peacock,0
Margaret Park,2
Steve Johnson,2


A query that uses multiple named subqueries in a WITH clause to gather total sales data on customers from India:

In [25]:
%%sql

WITH
    customers_india AS
        (
        SELECT * FROM customer
        WHERE country = "India"
        ),
    sales_per_customer AS
        (
         SELECT
             customer_id, SUM(total) AS total
         FROM invoice
         GROUP BY customer_id
        )

SELECT
    (ci.first_name || " " || ci.last_name) AS customer_name,
    ROUND(spc.total,2) AS total_purchases
FROM customers_india ci
INNER JOIN sales_per_customer AS spc 
    ON ci.customer_id = spc.customer_id
ORDER BY 1;

Done.


customer_name,total_purchases
Manoj Pareek,111.87
Puja Srivastava,71.28


A query to find the customer from each country that has spent the most money at our store, ordered alphabetically by country.

In [68]:
%%sql

WITH
    total_purchases AS
    (
    SELECT
        (c.first_name||" "||c.last_name) AS customer_name,
        billing_country AS country,
        ROUND(SUM(total),2) AS total_purchased
    FROM invoice AS i
    LEFT JOIN customer AS c ON c.customer_id=i.customer_id
    GROUP BY c.customer_id
    )

SELECT 
    country, 
    customer_name,
    MAX(total_purchased) AS total_purchased
FROM total_purchases
GROUP BY country

Done.


country,customer_name,total_purchased
Argentina,Diego Gutiérrez,39.6
Australia,Mark Taylor,81.18
Austria,Astrid Gruber,69.3
Belgium,Daan Peeters,60.39
Brazil,Luís Gonçalves,108.9
Canada,François Tremblay,99.99
Chile,Luis Rojas,97.02
Czech Republic,František Wichterlová,144.54
Denmark,Kara Nielsen,37.62
Finland,Terhi Hämäläinen,79.2


# Selecting New Albums To Purchase

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

You'll need to write a query to find out which genres sell the most tracks in the USA, write up a summary of your findings, and make a recommendation for the three artists whose albums we should purchase for the store.

In [30]:
%%sql

WITH
    track_genre AS
    (
    SELECT 
        t.track_id,
        t.name AS track_name,
        g.name AS genre_name
    FROM track AS t
    LEFT JOIN genre AS g
        ON g.genre_id=t.genre_id
    ),
    usa_tracks_sold AS
    (
    SELECT il.* FROM invoice_line AS il
    INNER JOIN invoice AS i on il.invoice_id = i.invoice_id
    INNER JOIN customer AS c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
    )

SELECT
    tg.genre_name, 
    SUM(uts.quantity) AS total_tracks,
    ROUND(CAST(SUM(uts.quantity) AS FLOAT)/(SELECT COUNT(*) FROM usa_tracks_sold),2) 
        AS percentage_sold 
FROM usa_tracks_sold AS uts
LEFT JOIN track_genre AS tg
    ON tg.track_id=uts.track_id
GROUP BY genre_name
ORDER BY 2 DESC


Done.


genre_name,total_tracks,percentage_sold
Rock,561,0.53
Alternative & Punk,130,0.12
Metal,124,0.12
R&B/Soul,53,0.05
Blues,36,0.03
Alternative,35,0.03
Latin,22,0.02
Pop,22,0.02
Hip Hop/Rap,20,0.02
Jazz,14,0.01


Recommendation: based on the current tracks sold, the investment should be focus on: 
1. Red Tone (Punk)
2. Slim Jim (Blues)
3. Meteor and the Girls (Pop)

Since Rock makes up 53% of total sales we should also recommend to increase the number of Rock artist

# Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

You might like to consider whether any extra columns from the employee table explain any variance you see, or whether the variance might instead be indicative of employee performance.

In [20]:
%%sql

WITH sales_employee AS
    (
    SELECT
        employee_id,
        (first_name||" "||last_name) AS employee_name,
        date('now')-birthdate AS age,
        date('now')-hire_date AS seniority
    FROM employee
    WHERE title LIKE "%Sales%"
    )

SELECT 
    se.employee_name,
    ROUND(SUM(i.total),1) AS total_sales,
    se.age, se.seniority
FROM invoice AS i
LEFT JOIN customer AS c ON c.customer_id = i.customer_id
LEFT JOIN sales_employee AS se ON se.employee_id = c.support_rep_id
GROUP BY se.employee_name

Done.


employee_name,total_sales,age,seniority
Jane Peacock,1731.5,48,4
Margaret Park,1584.0,74,4
Steve Johnson,1393.9,56,4


Jane seems to overperform the other two sales reps. Based on their seniority in the business (in years) we cannot deduct a significant factor. Absolute age does not explain the difference either so more investigation is needed. 

# Analzing Sales By Country

Your next task is to analyze the sales data for customers from each different country. You have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, you have been directed to calculate data, for each country, on the:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

In [64]:
%%sql

WITH invoice_country AS
    (
    SELECT i.*, c.country
    FROM invoice AS i
    LEFT JOIN customer AS c ON c.customer_id=i.customer_id
    ),
    country_sales AS
    (
    SELECT 
        country,
        ROUND(SUM(total),2) AS total_sales
    FROM invoice_country
    GROUP BY country ORDER BY 2 DESC
    ),
    country_aov AS
    (
    SELECT 
        country,
        ROUND(AVG(total),2) AS average_order_value
    FROM invoice_country
    GROUP BY country ORDER BY 2 DESC
    ),
    customers_per_country AS
    (
    SELECT 
        country,
        COUNT(DISTINCT customer_id) AS total_customers
    FROM invoice_country
    GROUP BY country ORDER BY 2 DESC
    )

SELECT
    cs.country, 
    cs.total_sales,
    ca.average_order_value AS AOV,
    cpc.total_customers,
    ROUND(cs.total_sales/cpc.total_customers,2) AS avg_per_customer
FROM country_sales AS cs
LEFT JOIN country_aov AS ca ON ca.country=cs.country
LEFT JOIN customers_per_country AS cpc ON cpc.country=cs.country
ORDER BY 2 DESC
LIMIT 10

Done.


country,total_sales,AOV,total_customers,avg_per_customer
USA,1040.49,7.94,13,80.04
Canada,535.59,7.05,8,66.95
Brazil,427.68,7.01,5,85.54
France,389.07,7.78,5,77.81
Germany,334.62,8.16,4,83.66
Czech Republic,273.24,9.11,2,136.62
United Kingdom,245.52,8.77,3,81.84
Portugal,185.13,6.38,2,92.56
India,183.15,8.72,2,91.58
Ireland,114.84,8.83,1,114.84


Alternative solution...

In [67]:

%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        ROUND(SUM(unit_price),1) total_sales,
        ROUND(SUM(unit_price) / count(distinct customer_id),1) customer_lifetime_value,
        ROUND(SUM(unit_price) / count(distinct invoice_id),1) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.5,7.9,80.0
Canada,8,535.6,7.0,66.9
Brazil,5,427.7,7.0,85.5
France,5,389.1,7.8,77.8
Germany,4,334.6,8.2,83.7
Czech Republic,2,273.2,9.1,136.6
United Kingdom,3,245.5,8.8,81.8
Portugal,2,185.1,6.4,92.6
India,2,183.2,8.7,91.6
Other,15,1094.9,7.4,73.0


# Albums vs Individual Tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

- Purchase a whole album
- Purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

Write a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:
- Number of invoices
- Percentage of invoices

In [12]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (SELECT COUNT(*) FROM invoice) AS percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.

# Additional Ideas

- Which artist is used in the most playlists?
- How many tracks have been purchased vs not purchased?
- Is the range of tracks in the store reflective of their sales popularity?
- Do protected vs non-protected media types have an effect on popularity?